In [75]:
import pandas as pd
import re
import json
import numpy as np

In [2]:
hn = pd.read_csv('hacker_news.csv')

In [3]:
hn.head()

,id,title,url,num_points,num_comments,author,created_at
0,12224879,Interactive Dynamic Video,http://www.interactivedynamicvideo.com/,386,52,ne0phyte,8/4/2016 11:52
1,11964716,Florida DJs May Face Felony for April Fools' W...,http://www.thewire.com/entertainment/2013/04/f...,2,1,vezycash,6/23/2016 22:20
2,11919867,Technology ventures: From Idea to Enterprise,https://www.amazon.com/Technology-Ventures-Ent...,3,1,hswarna,6/17/2016 0:01
3,10301696,Note by Note: The Making of Steinway L1037 (2007),http://www.nytimes.com/2007/11/07/movies/07ste...,8,2,walterbell,9/30/2015 4:12
4,10482257,Title II kills investment? Comcast and other I...,http://arstechnica.com/business/2015/10/comcas...,53,22,Deinos,10/31/2015 9:48


In [4]:
titles = hn['title']

In [5]:
pattern = r'[Pp]ython'
titles.str.contains(pattern).sum()

160

In [6]:
titles.str.contains(r'python', flags=re.I).sum()

160

In [7]:
pat_sql = r'[Ss][Qq][Ll]'

In [8]:
sql_counts = titles.str.contains(r'sql',flags=re.I).sum()
sql_counts

108

In [9]:
pattern = r'(\w+SQL)'

In [10]:
sql_counts = titles.str.extract(pattern,flags=re.I, expand=False).value_counts()
sql_counts

PostgreSQL    27
NoSQL         16
MySQL         12
nosql          1
mySql          1
SparkSQL       1
MemSQL         1
CloudSQL       1
Name: title, dtype: int64

In [11]:
hn_sql = hn[hn['title'].str.contains(r"\w+SQL", flags=re.I)].copy()

In [12]:
hn_sql['flavor']=hn_sql['title'].str.extract(r"(\w+SQL)", flags=re.I, expand=False).copy().str.lower()
sql_pivot = hn_sql.pivot_table(index='flavor', values = 'num_comments', aggfunc='mean')
sql_pivot

,num_comments
flavor,
cloudsql,5.000000
memsql,14.000000
mysql,12.230769
nosql,14.529412
postgresql,25.962963
sparksql,1.000000


In [13]:
a = pd.Series([1,2,3,4,5])
b = pd.Series([7,8,9,0,12])
c = dict(zip(a,b))
c

{1: 7, 2: 8, 3: 9, 4: 0, 5: 12}

Write a regular expression pattern which will match Python or python, followed by a space, followed by one or more digit characters or periods.
The regular expression should contain a capture group for the digit and period characters (the Python versions)
Extract the Python versions from titles using the regular expression pattern.
Use Series.value_counts() and the dict() function to create a dictionary frequency table of the extracted Python versions. Assign the result to py_versions_freq.

In [14]:
pattern = r"[Pp]ython ?([\d\.]+)"
py_versions = titles.str.extract(pattern, flags=re.I, expand=False).value_counts()
py_versions_freq = dict(py_versions)
py_versions_freq

{'3': 10,
 '3.5': 4,
 '2': 3,
 '4': 2,
 '3.6': 2,
 '3.5.0': 1,
 '2.7': 1,
 '8': 1,
 '1.5': 1}

In [15]:
def first_10_matches(pattern):
    """
    Return the first 10 story titles that match
    the provided regular expression
    """
    all_matches = titles[titles.str.contains(pattern)]
    first_10 = all_matches.head(10)
    return first_10



In [16]:
pattern = r"(?<!\bSeries )\b[Cc]\b(?!\+)(?!\.)"
first_10_matches(pattern)
c_mentions = titles.str.contains(pattern).sum()
c_mentions

102

In [17]:
test_cases = ['Rred_Green_Bluue',
              'Yellow_Green_Red',
              'Red_Green_Red',
              'Yellow_Green_Blue',
              'Green']

In [18]:
def run_test_cases(pattern):
    for tc in test_cases:
        result = re.search(pattern, tc, flags=re.I)
        print(result or "No Match")

In [19]:
run_test_cases(r"Green(?=_Blue)")

No Match
No Match
No Match
<re.Match object; span=(7, 12), match='Green'>
No Match


In [20]:
run_test_cases(r"Green(?!_Red)")

<re.Match object; span=(5, 10), match='Green'>
No Match
No Match
<re.Match object; span=(7, 12), match='Green'>
<re.Match object; span=(0, 5), match='Green'>


In [21]:
run_test_cases(r"(?<=Red_)Green")

<re.Match object; span=(5, 10), match='Green'>
No Match
<re.Match object; span=(4, 9), match='Green'>
No Match
No Match


In [22]:
run_test_cases(r"(?<!Yellow_)Green")

<re.Match object; span=(5, 10), match='Green'>
No Match
<re.Match object; span=(4, 9), match='Green'>
No Match
<re.Match object; span=(0, 5), match='Green'>


In [23]:
run_test_cases(r"(\w)\1")

<re.Match object; span=(0, 2), match='Rr'>
<re.Match object; span=(2, 4), match='ll'>
<re.Match object; span=(6, 8), match='ee'>
<re.Match object; span=(2, 4), match='ll'>
<re.Match object; span=(2, 4), match='ee'>


# List Comprehensions and Lamda Functions

In [28]:
world_cup_str = """
[
    {
        "team_1": "France",
        "team_2": "Croatia",
        "game_type": "Final",
        "score" : [4, 2]
    },
    {
        "team_1": "Belgium",
        "team_2": "England",
        "game_type": "3rd/4th Playoff",
        "score" : [2, 0]
    }
]
"""

In [30]:
world_cup_obj = json.loads(world_cup_str)
world_cup_obj

[{'team_1': 'France',
  'team_2': 'Croatia',
  'game_type': 'Final',
  'score': [4, 2]},
 {'team_1': 'Belgium',
  'team_2': 'England',
  'game_type': '3rd/4th Playoff',
  'score': [2, 0]}]

In [139]:
file = open('hn_2014.json')
hn = json.load(file)

hn_1 = hn.copy()
hn_1[1]

{'author': 'jcr',
 'numComments': 0,
 'points': 1,
 'url': 'http://spectrum.ieee.org/automaton/robotics/home-robots/telemba-telepresence-robot',
 'storyText': '',
 'createdAt': '2014-05-29T08:05:58Z',
 'tags': ['story', 'author_jcr', 'story_7815234'],
 'createdAtI': 1401350758,
 'title': 'Telemba Turns Your Old Roomba and Tablet Into a Telepresence Robot',
 'objectId': '7815234'}

In [63]:
hn_clean = []

In [59]:
def del_key(dict_, key):
    # create a copy so we don't
    # modify the original dict
    modified_dict = dict_.copy()
    del modified_dict[key]
    return modified_dict

In [60]:
hn[0]

{'author': 'dragongraphics',
 'numComments': 0,
 'points': 2,
 'url': 'http://ashleynolan.co.uk/blog/are-we-getting-too-sassy',
 'storyText': '',
 'createdAt': '2014-05-29T08:07:50Z',
 'tags': ['story', 'author_dragongraphics', 'story_7815238'],
 'createdAtI': 1401350870,
 'title': 'Are we getting too Sassy? Weighing up micro-optimisation vs. maintainability',
 'objectId': '7815238'}

In [61]:
delete_key = 'createdAtI'
for i in hn:
    hn_clean.append(del_key(i, delete_key))

In [62]:
hn_clean[0]

{'author': 'dragongraphics',
 'numComments': 0,
 'points': 2,
 'url': 'http://ashleynolan.co.uk/blog/are-we-getting-too-sassy',
 'storyText': '',
 'createdAt': '2014-05-29T08:07:50Z',
 'tags': ['story', 'author_dragongraphics', 'story_7815238'],
 'title': 'Are we getting too Sassy? Weighing up micro-optimisation vs. maintainability',
 'objectId': '7815238'}

In [104]:
hn_clean = [del_key(i,delete_key) for i in hn]

In [71]:
'col_{0} and {1}'.format(1,6)

'col_1 and 6'

In [ ]:
col = []
for i in range (1,5):
    col.append('col{}'.format(i))

In [74]:
cols = ['col_{}'.format(i) for i in range(1,5)]
cols

['col_1', 'col_2', 'col_3', 'col_4']

In [79]:
data = np.zeros((4,4))
data_frame = pd.DataFrame(data = data , columns = cols)
data_frame

,col_1,col_2,col_3,col_4
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0


In [94]:
for i,k in enumerate(hn):
    #print(hn[i]['url'])


SyntaxError: unexpected EOF while parsing (<ipython-input-94-d02db1aafc50>, line 2)

In [100]:
urls = [d['url'] for d in hn]

In [107]:
has_comments = []
has_comments = [d['title'] for d in hn_clean if d['numComments']>0]

In [109]:
len(has_comments)

9279

In [115]:
thousand_points = []
thousand_points = [d['points'] for d in hn_clean if d['points']>1000]
num_thousand_points = min(thousand_points)
num_thousand_points

1054

In [122]:
def get_comments(dict_item):
    return dict_item['numComments']

most_comments = max(hn_clean, key=lambda a:a['numComments'])
    

In [123]:
print(most_comments)

{'author': 'platz', 'numComments': 1208, 'points': 889, 'url': 'https://blog.mozilla.org/blog/2014/04/03/brendan-eich-steps-down-as-mozilla-ceo/', 'storyText': None, 'createdAt': '2014-04-03T19:02:53Z', 'tags': ['story', 'author_platz', 'story_7525198'], 'title': 'Brendan Eich Steps Down as Mozilla CEO', 'objectId': '7525198'}


In [119]:
multiply = lambda a,b:a*b

In [135]:
hn_sorted_points = sorted (hn_clean, key = lambda x : x['points'], reverse=True)
hn_sorted_points[0:4]

[{'author': 'frederfred',
  'numComments': 398,
  'points': 2732,
  'url': 'http://gabrielecirulli.github.io/2048/',
  'storyText': '',
  'createdAt': '2014-03-10T15:44:42Z',
  'tags': ['story', 'author_frederfred', 'story_7373566'],
  'title': '2048',
  'objectId': '7373566'},
 {'author': 'brokenparser',
  'numComments': 260,
  'points': 1958,
  'url': 'https://thedaywefightback.org/',
  'storyText': '',
  'createdAt': '2014-02-11T08:12:28Z',
  'tags': ['story', 'author_brokenparser', 'story_7216471'],
  'title': 'Today is The Day We Fight Back',
  'objectId': '7216471'},
 {'author': 'jamesbritt',
  'numComments': 308,
  'points': 1522,
  'url': 'https://plus.google.com/+CarmsPerez/posts/GnVTvQNgvpf',
  'storyText': None,
  'createdAt': '2014-01-19T22:46:05Z',
  'tags': ['story', 'author_jamesbritt', 'story_7086411'],
  'title': 'Wozniak: “Actually, the movie was largely a lie about me”',
  'objectId': '7086411'},
 {'author': 'keithwarren',
  'numComments': 451,
  'points': 1297,
  'u

In [136]:
top_5_titles = [d['title'] for d in hn_sorted_points[0:5]]
top_5_titles

['2048',
 'Today is The Day We Fight Back',
 'Wozniak: “Actually, the movie was largely a lie about me”',
 'Microsoft Open Sources C# Compiler',
 'Elon Musk: To the People of New Jersey']

In [181]:
hn_df=pd.DataFrame(hn_clean)
four_tags = hn_df['tags'][hn_df['tags'].apply(len)==4]
four_tags.head(4)

43     [story, author_alamgir_mand, story_7813869, sh...
86       [story, author_cweagans, story_7812404, ask_hn]
104    [story, author_nightstrike789, story_7812099, ...
107    [story, author_ISeemToBeAVerb, story_7812048, ...
Name: tags, dtype: object

In [159]:
lambda l: l[-1] if len(l)==4 else none

<function __main__.<lambda>(l)>

In [182]:
cleaned_tags = []
cleaned_tags = hn_df['tags'].apply((lambda l: l[-1] if len(l)==4 else None))


In [183]:
hn_df['tags'] = cleaned_tags